# Interactive Quantum Mechanics Examples

This notebook contains interactive examples of different quantum mechanical situations. 

In [ ]:
## make sure numpy and matplotlib are installed
!pip install numpy matplotlib

In [78]:
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import figure
from ipywidgets import interact, FloatSlider, IntSlider

hbarc = 197.3269804 ## in units [MeV] [fm]

## Part 1: The step potential

For the step potential there are two regions:
   - Region 1: $V = 0$ for $x<0$
   - Region 2: $V = V_0$ for $x>0$
   
The general solutions to the Schrodinger equation can be written in the form 
$$
\begin{align}
\psi_1(x) &= A e^{ik_1x} + B e^{-ik_1x}\\
\psi_2(x) &= C e^{ik_2x} + D e^{-ik_2x},
\end{align}
$$
where the wave numbers are given by
$$
\begin{align}
k_{1}  &= \sqrt{\frac{2m(E)}{\hbar^2}}\\
k_{2}  &= \sqrt{\frac{2m(E - V)}{\hbar^2}}
\end{align}
$$


In the course notes you will have seen that the solutions can be written in the form  
$$
\begin{align}
\psi_1(x) &= e^{ik_1x} + r e^{-ik_1x}\\
\psi_2(x) &= t e^{ik_2x},
\end{align}
$$ 
where $r$ and $t$ are given by 
$$
\begin{align}
r &= \frac{k_1-k_2}{k_1+k_2}\\
t &= \frac{2k_1}{k_1+k_2}.
\end{align}
$$
We will now visualize these results interactively. 

In [79]:
## Define the wavefunction as it's own function so we can reuse it
def calc_step_wavefunction(x,V,E,m):
    # Calculate the wave numbers
    k_1 = np.sqrt(2*m*(1.+0.j)*E/hbarc**2)
    k_2 = np.sqrt(2*m*(1.+0.j)*(E-V)/hbarc**2) 
     
    r = (k_1-k_2)/(k_1+k_2)
    t = 2*k_1/(k_1+k_2)

    psi_1_plus  =   np.exp( 1.j*x*k_1)*(x<0.0)
    psi_1_minus = r*np.exp(-1.j*x*k_1)*(x<0.0)
    psi_2_plus  = t*np.exp( 1.j*x*k_2)*(x>0.0) 
    
    wavefunction = psi_1_plus + psi_1_minus + psi_2_plus
    
    return {"total"       : wavefunction,
            "psi_1_plus"  : psi_1_plus,
            "psi_1_minus" : psi_1_minus,
            "psi_2_plus"  : psi_2_plus,
            "t"           : t,
            "r"           : r}

In [80]:
m = FloatSlider(value = 0.5, min=0.01, max=5.0, steps=0.1,description = 'm [MeV/$c^2$]')
E = FloatSlider(value = 1.0, min=0.01, max=5.0, steps=0.1,description = 'E [MeV]')
V = FloatSlider(value = 2.0, min=0.01, max=5.0, steps=0.1,description = 'V [MeV]')
@interact
def step_potential(V=V,E=E,m=m):
    # Set up the x axis
    min_val = -2000.0
    max_val =  2000.0
    x = np.linspace(min_val, max_val, 1000)

    # Draw the potential function 
    potential=V*(x>0.0)
    energy = E*(x<=max_val)
    
    # Calculate the wavefunction
    wavefunction = calc_step_wavefunction(x,V,E,m)
    R = np.abs(wavefunction['r'])**2
    # Plot the real and imaginary parts
    fig, ax = plt.subplots(1, 1,figsize=(10,5))
    ax.plot(x, np.real(wavefunction['total']),color='lightblue',label='Real part of $\psi(x)$')
    ax.plot(x, np.imag(wavefunction['total']),color='lightblue',linestyle='--',label= f'Imaginary part of $\psi(x)$')
    ax.set_ylabel('$\psi(x)$')
    ax.set_xlabel('x [fm]')
    ax.set_ylim([-5, 5])
    ax.set_xlim([min_val, max_val])
    ax.legend(loc='upper left')
    
    ax2=ax.twinx()
    ax2.set_ylabel('Energy [MeV]')
    ax2.plot(x, potential,label= 'Potential energy')
    ax2.plot(x, energy,label= 'Total energy')
    ax2.set_ylim([-5, 5])
    ax2.set_xlim([min_val, max_val])
    ax2.legend(loc='upper right')
    return plt.figure()

interactive(children=(FloatSlider(value=2.0, description='V [MeV]', max=5.0, min=0.01), FloatSlider(value=1.0,…

### Avenues of investigation 
Here you have a chance to become familiar with the solutions to schrodinger equation. 
Investigate how the solution changes as you change the parameter values:
- Describe how the wave function changes when the particle energy $E$ and the potential height $V_0$ are varied. Investigate for both $E>V_0$ and $E<V_0$.
- When $E<V_0$, how does the potential height and particle mass affect the amplitude of the exponentially falling wavefunction inside the barrier? 

### The reflection and transmission coefficiencts
We can also plot how the reflection and transmission coefficiencts vary as a function of the energy.
The probability that the particle reflects off the potential step is given by the squared magnitude of $r$
$$
R = |r|^2
$$
Similarly, the probability that the particle is transmitted is given by $T = 1 - R$ because these two probabilities must sum to 1, i.e. the particle has to go *somewhere*.  

In [81]:
m = FloatSlider(value = 0.5, min=0.01, max=5.0, steps=0.1,description = 'm [MeV/$c^2$]')
V = FloatSlider(value = 2.0, min=0.01, max=5.0, steps=0.1,description = 'V [MeV]')
@interact
def step_transmission(V=V,m=m):
    ## Set up the 
    min_val =  0.0
    max_val =  3.1
    E = np.linspace(min_val, max_val, 1000)
    classical = (E>V)
    wavefunction = calc_step_wavefunction(1,V,E,m)
    
    ## Create reflection and transmission probability
    R = np.abs(wavefunction['r'])**2
    T = 1-R 
    
    fig, ax = plt.subplots(1, 2,figsize=(10,5))
    ax[0].plot(E, T)
    ax[0].plot(E, classical, color='black', linestyle='--')
    ax[0].set_ylabel('Transmission probability')
    ax[0].set_xlabel('E')
    ax[0].set_ylim([0, 1.1])
    
    ax[1].plot(E, R)
    ax[1].plot(E, (E<V), color='black', linestyle='--')
    ax[1].set_ylabel('Reflection probability')
    ax[1].set_xlabel('E')
    ax[1].set_ylim([0, 1.1])

    return plt.figure()

interactive(children=(FloatSlider(value=2.0, description='V [MeV]', max=5.0, min=0.01), FloatSlider(value=0.5,…

### Investigating reflection and transmission



## Part 2: The square barrier potential 

For the square barrier potential we create three regions, allowing the w
$$
\begin{align}
\psi_{1}(x) &= A e^{ik_{out}x} + B e^{-ik_{out}x}  \\
\psi_{2}(x) &= C e^{ik_{in}x} + D e^{-ik_{in}x} \\
\psi_{3}(x) &= E e^{ik_{in}x}
\end{align}
$$

Where

$$
k_{out} = \sqrt{\frac{2m(E)}{\hbar^2}}
$$


$$
k_{in} = \sqrt{\frac{2m(E - V)}{\hbar^2}}
$$

If $E<V$ $k_{in}$ becomes imaginary, changing the wavefunction to the sum of two exponetials. 
Rewrite it as $ k_{in} = i q$ so the wavefunction becomes 
$$
\psi_{2}(x) = C e^{-qx} + D e^{qx}
$$

We can measure the various components relative to the coefficienct $A$.  

The wave function must be continuous so we have the following boundary conditions:

$$
\begin{align}
\psi_{1}(0) &= \psi_{2}(0)\\
\frac{d\psi_{1}(0)}{dx} &= \frac{d\psi_{2}(0)}{dx}\\
\psi_{2}(a) &= \psi_{3}(a)\\
\frac{d\psi_{2}(a)}{dx} &= \frac{d\psi_{3}(a)}{dx}
\end{align}
$$

This gives us the following set of simultanious equations
$$
\begin{align}
A          + B         &= C + D\\
ik_{out}A   - ik_{out}B &= ik_{in}C -ik_{in} D\\
C e^{ik_{in}a}  + D e^{-ik_{in}a}  &= E e^{ik_{out}a}\\
ik_{in}C e^{ik_{in}a}-ik_{in}D e^{-ik_{in}a} &= ik_{out}E e^{ik_{out}a}
\end{align}
$$

Now we can set $A = 1$, $B=r$ and $C=t$ to represent the incoming, reclected and transmitted
$$
\begin{align}
1          + r         &= C + D\\
ik_{out}   - ik_{out}r &= ik_{in}C -ik_{in} D\\
C e^{ik_{in}a}  + D e^{-ik_{in}a}  &= t e^{ik_{out}a}\\
ik_{in}C e^{ik_{in}a}-ik_{in}D e^{-ik_{in}a} &= ik_{out}t e^{ik_{out}a}
\end{align}
$$

Rearranging the second equation gives us
$$
\frac{k_{out}}{k_{in}}(1-r) = C-D
$$
Now, adding and/or substracting this with the first equation gives us

$$
\begin{align}
2C &=1+r+\frac{k_{out}}{k_{in}}(1-r) \\
2D &=1+r-\frac{k_{out}}{k_{in}}(1-r) \\
\end{align}
$$
Simiarly, the fourth equation can be rearranged to give
$$
C e^{ik_{in}a} - D e^{-ik_{in}a} = \frac{k_{out}} {k_{in}} t e^{ik_{out}a}
$$

which can be added and subtracted to the third equation to give

$$
\begin{align}
2C e^{ik_{in}a}&= t e^{ik_{out}a} + \frac{k_{out}}{k_{in}}t e^{ik_{out}a}\\
2D e^{-ik_{in}a}&= t e^{ik_{out}a} - \frac{k_{out}}{k_{in}}t e^{ik_{out}a}\\
\end{align}
$$

We can show that

$$
r = \frac{(k_{out}^2-k_{in}^2)\sin(ak_{in})}{2ik_{out}k_{in}\cos(ak_{in}) +(k_{out}^2+k_{in}^2)\sin(ak_{in}) }
$$

$$
t = \frac{4 k_{in} k_{out} e^{-ia(k_{out}-k_{in})} }{ (k_{out}+k_{in})^2 - e^{2iak_{in}}(k_{out}-k_{in})^2 } 
$$

In [82]:
## Define the wavefunction as it's own function so we can reuse it

def calc_wavefunction(x,a,V,E,m):
    # Calculate the wave numbers
    k_out = np.sqrt(2*m*(1.+0.j)*E/hbarc**2)
    k_in  = np.sqrt(2*m*(1.+0.j)*(E-V)/hbarc**2)
    
    r = ((k_out*k_out-k_in*k_in)*np.sin(a*k_in)) / (2.j*k_out * k_in * np.cos(a*k_in) + (k_out*k_out + k_in*k_in) * np.sin(a*k_in))
    t = (4*k_in*k_out*np.exp(-1.j*a*(k_out-k_in)) ) / ((k_out+k_in)**2 - np.exp(2.j*a*k_in)*(k_out-k_in)**2)
    
    C = 0.5*(1+r+(k_out/k_in)*(1-r) )
    D = 0.5*(1+r-(k_out/k_in)*(1-r) )
    
    fcn_r1_plus  =   np.exp( 1.j*x*k_out)*(x<0.0)
    fcn_r1_minus = r*np.exp(-1.j*x*k_out)*(x<0.0)
    fcn_r2_plus  = C*np.exp( 1.j*x*k_in) *((x>0.0)& (x<a)) 
    fcn_r2_minus = D*np.exp(-1.j*x*k_in) *((x>0.0)& (x<a))
    fcn_r3_plus  = t*np.exp( 1.j*x*k_out)*(x>a)
    
    wavefunction = fcn_r1_plus + fcn_r1_minus + fcn_r2_plus + fcn_r2_minus + fcn_r3_plus 
    
    return {"total"    : wavefunction,
            "r1_plus"  : fcn_r1_plus,
            "r1_minus" : fcn_r1_minus,
            "r2_plus"  : fcn_r2_plus,
            "r2_minus" : fcn_r2_minus,
            "r3_plus"  : fcn_r3_plus,
            "t"        : t,
            "r"        : r}


In [83]:
a = FloatSlider(value = 100.0, min=0.01, max=1000.0, steps=0.1,description = 'a [fm]')
E = FloatSlider(value = 1.0, min=0.01, max=5.0, steps=0.1,description = 'E [MeV]')
V = FloatSlider(value = 2.0, min=-2.0, max=5.0, steps=0.1,description = 'V [MeV]')
m = FloatSlider(value = 0.5, min=0.01, max=5.0, steps=0.1,description = 'm [MeV/$c^2$]')
@interact
def square_barrier(a=a,V=V,E=E,m=m):
    # Set up the x axis
    min_val = -2000.0
    max_val =  2000.0
    x = np.linspace(min_val, max_val, 1000)

    # Draw the potential function 
    potential=V*( (x>0.0)& (x<a))
    energy = E*(x<=max_val)
    
    # Calculate the wavefunction
    wavefunction = calc_wavefunction(x,a,V,E,m)
    
    # Plot the real and imaginary parts
    fig, ax = plt.subplots(1, 1,figsize=(10,5))
    ax.plot(x, np.real(wavefunction['total']),color='lightblue')
    ax.plot(x, np.imag(wavefunction['total']),color='lightblue',linestyle='--')
    ax2=ax.twinx()
    ax2.set_ylabel('Energy [MeV]')
    ax.plot(x, potential)
    ax.plot(x, energy)
    ax.set_ylabel('$\psi(x)$')
    ax.set_xlabel('x [fm]')
    ax.set_ylim([-5, 5])

    return plt.figure()

interactive(children=(FloatSlider(value=100.0, description='a [fm]', max=1000.0, min=0.01), FloatSlider(value=…

In [84]:
a = FloatSlider(value = 100.0, min=0.01, max=1000.0, steps=0.1)
m = FloatSlider(value = 0.5, min=0.01, max=5.0, steps=0.1,description = 'm [MeV/$c^2$]')
@interact
def transmission(a=a,m=m):
    ## Set up the 
    min_val =  0.0
    max_val =  3.1
    E = np.linspace(min_val, max_val, 1000)
    V = 1.0
    classical = (E>V)
    wavefunction = calc_wavefunction(a+1,a,V,E,m)
    
    fig, ax = plt.subplots(1, 2,figsize=(10,5))
    
    ax[0].plot(E, np.abs(wavefunction['t']*wavefunction['t']))
    ax[0].plot(E, classical, color='black', linestyle='--')
    ax[0].set_ylabel('Transmission probability')
    ax[0].set_xlabel('E')
    ax[0].set_ylim([0, 1.1])
    
    ax[1].plot(E, np.abs(wavefunction['r']*wavefunction['r']))
    ax[1].plot(E, (E<V), color='black', linestyle='--')
    ax[1].set_ylabel('Reflection probability')
    ax[1].set_xlabel('E')
    ax[1].set_ylim([0, 1.1])

    return plt.figure()

interactive(children=(FloatSlider(value=100.0, description='a', max=1000.0, min=0.01), FloatSlider(value=0.5, …